In [1]:
from datetime import datetime
import requests


In [2]:
# Datetime ISO 8601 Format to Timestamp, TZ='America/Bogota' -05:00
def to_timestamp(datetime_iso8601):
    # datetime_iso8601 = '2023-03-17T00:00:00-05:00'
    return int(datetime.fromisoformat(datetime_iso8601).timestamp() * 1000)

In [3]:
# Datetime ISO 8601 Format to Timestamp
#to_timestamp('2023-03-17T00:00:00-05:00')

In [4]:
# Request to InfluxDB API REST
def request_influxdb(sql_query):
    endpoint = "http://influxdb.canair.io:8086/query"
    database = "canairio"
    parameters = {
        'db': database,
        'q': sql_query,
        'epoch': 'ms'
    }
    # To get response as CSV text
    headers = {'Accept': 'application/csv'}
    # GET Request
    return requests.get(endpoint, params=parameters, headers=headers)

In [5]:
# Query Tangaras
def query_tangaras(start_timestamp, end_timestamp):
    # Period DateTime
    period_time = f"time >= {start_timestamp}ms AND time <= {end_timestamp}ms"
    # SQL
    sql_query = "SELECT DISTINCT(geo) AS \"geohash\" "\
                "FROM \"fixed_stations_01\" WHERE "\
                "(\"geo3\" = 'd29') AND "\
                f"{period_time} "\
                "GROUP BY \"name\";"
    
    return sql_query

In [6]:
# Query Data Tangaras
def query_data(tangaras, start_timestamp, end_timestamp, datatype='pm25', time='30s'):
    # datatype = ['pm25', 'tmp', 'hum']
    # time = ['30s', '1m', '1h', '24h', '7d', '30d']
    # Period DateTime
    period_time = f"time >= {start_timestamp}ms AND time <= {end_timestamp}ms"
    # SQL Datatype by Tangara Sensor
    sql_query = ""
    for mac in tangaras['MAC'].to_list():
        sql_query += f"SELECT \"name\", last(\"{datatype}\") "\
                    "FROM \"fixed_stations_01\" WHERE "\
                    f"(\"name\" = '{mac}') AND "\
                    f"{period_time} " \
                    f"GROUP BY time({time}) fill(none); "
    return sql_query[:-2]